# Задача

Задача: создать систему оценки текстов

Решение: регрессия

Результат: 0.361 MSE

# input

In [3]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from torch.nn.functional import mse_loss
from sklearn.metrics import mean_squared_error

In [4]:
RANDOM_STATE = 101

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from tqdm import tqdm
RANDOM_STATE = 101
from sklearn.ensemble import RandomForestClassifier
#from imblearn.pipeline import Pipeline, make_pipeline
#from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import roc_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error 
from sklearn.dummy import DummyClassifier
#pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_recall_curve, classification_report

In [6]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Загрузка данных

In [7]:
train = pd.read_csv(r"C:\Users\Dmitri\Downloads\NLP\грамматика\train.csv")

In [8]:
test = pd.read_csv(r"C:\Users\Dmitri\Downloads\NLP\грамматика\test.csv")

In [9]:
train.describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
count,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000
mean,3.127077,3.028254,3.235745,3.116850,3.032856,3.081053
std,0.662542,0.644399,0.583148,0.655997,0.699841,0.671450
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text_id      3911 non-null   object 
 1   full_text    3911 non-null   object 
 2   cohesion     3911 non-null   float64
 3   syntax       3911 non-null   float64
 4   vocabulary   3911 non-null   float64
 5   phraseology  3911 non-null   float64
 6   grammar      3911 non-null   float64
 7   conventions  3911 non-null   float64
dtypes: float64(6), object(2)
memory usage: 244.6+ KB


In [11]:
train

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [12]:
train.columns

Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions'],
      dtype='object')

In [13]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [14]:
X = train['full_text']
y = train[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=RANDOM_STATE)

In [16]:
y_train

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
2923,3.0,2.0,3.0,3.0,2.5,3.0
2978,2.5,2.5,2.0,2.5,2.0,2.0
1850,2.0,2.0,3.5,3.0,3.0,3.5
2796,4.0,4.0,3.5,3.5,3.5,4.0
3075,4.0,4.5,5.0,5.0,5.0,4.0
...,...,...,...,...,...,...
599,4.0,4.5,4.0,4.0,4.5,5.0
1599,3.5,3.0,3.5,3.5,4.0,3.5
1361,3.5,3.5,4.0,3.0,4.0,4.0
1547,2.0,2.5,2.5,2.0,2.0,2.0


# Извлечение признаков

In [17]:
%%time
vectorizer = TfidfVectorizer() 
vectorizer.fit(X_train)

CPU times: total: 797 ms
Wall time: 937 ms


TfidfVectorizer()

In [18]:
def tf(text, vectorizer=vectorizer, fit=False):
    if type(text) == str:
        text = text.split("\n")
    if fit==True:
        tfidf_matrix = vectorizer.fit_transform(text)
    else:
        tfidf_matrix = vectorizer.transform(text)
    return tfidf_matrix.toarray().astype(np.float16)

In [19]:
X_train_tf = tf(X_train)
X_test_tf = tf(X_test)

In [20]:
X_train_tf.shape, X_test_tf.shape

((2933, 18343), (978, 18343))

# Обучение

In [21]:
from torch.utils.data import DataLoader, TensorDataset

# Создайте TensorDataset для обучения и валидации
train_dataset = TensorDataset(torch.tensor(X_train_tf, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_test_tf, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))

# Создайте DataLoader с параметрами batch_size и shuffle
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, drop_last=True)

In [22]:
for i in train_loader:
    break

In [23]:
i[0].shape, i[1].shape

(torch.Size([64, 18343]), torch.Size([64, 6]))

In [24]:
class MultiOutputRegressor(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        loss = self.loss_fn(pred, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        loss = self.loss_fn(pred, y)
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [25]:
# Пример использования
input_dim = 18343
output_dim = 6
epochs = 10
batch_size = 64

In [26]:
# Инициализация модели
model = MultiOutputRegressor(input_dim, output_dim)

In [27]:
# Обучение модели с использованием PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=epochs)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
trainer.fit(model, train_loader, val_loader)

Missing logger folder: C:\Users\Dmitri\Downloads\NLP\grammar\lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | net     | Sequential | 1.2 M 
1 | loss_fn | MSELoss    | 0     
---------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.698     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Dmitri\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Dmitri\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Dmitri\anaconda3\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# Валидация

In [85]:
answ = list(map(list, zip(*torch.tensor(y_test.values, dtype=torch.float32))))

In [86]:
pred = list(map(list, zip(*model(torch.tensor(X_test_tf, dtype=torch.float32)))))

In [87]:
def round_half(tensor, be_tensor=True):
    tensor = float(tensor)
    after_part = tensor%1
    if after_part>0.5:
        if after_part>0.75:
            after_part = 1
        else:
            after_part = 0.5
    else:
        if after_part>0.25:
            after_part = 0.5
        else:
            after_part = 0
    if be_tensor:
        return torch.tensor(tensor//1+after_part)
    else:
        return tensor//1+after_part

In [88]:
pred = [[round_half(tensor) for tensor in tensors] for tensors in pred]

In [89]:
answ = [[round_half(tensor) for tensor in tensors] for tensors in answ]

In [90]:
result = []
for i in range(6):
    x_tensor = torch.stack(answ[i])
    y_tensor = torch.stack(pred[i])
    mse = round(float(mse_loss(y_tensor, x_tensor)), 4)
    result.append(mse)
    print(mse)

0.3648
0.3548
0.2935
0.341
0.4011
0.4105


In [95]:
np.round(np.mean(result), 4)

0.361

# Предсказание

In [78]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [79]:
pred = list(map(list, zip(*model(torch.tensor(tf(test['full_text']), dtype=torch.float32)))))

In [81]:
pred = [[round_half(tensor, be_tensor=False) for tensor in tensors] for tensors in pred]

In [82]:
pred

[[3.0, 2.5, 3.5],
 [3.0, 2.5, 3.5],
 [3.5, 2.5, 3.5],
 [3.0, 2.5, 3.5],
 [3.0, 2.5, 3.5],
 [3.0, 2.5, 3.5]]